## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/120-googleq-a-train-3fold-bert-base-unc-host/' + '*.h5')
model_path_list.sort()
print('Models to predict:', model_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/120-googleq-a-train-3fold-bert-base-unc-host/model_fold_1.h5', '/kaggle/input/120-googleq-a-train-3fold-bert-base-unc-host/model_fold_2.h5', '/kaggle/input/120-googleq-a-train-3fold-bert-base-unc-host/model_fold_3.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
    
for feature in text_features:
    # Lower
    test[feature] = test[feature].apply(lambda x: x.lower())
    # Map misspellings
    test[feature] = test[feature].apply(lambda x: map_misspellings(x))
    # Map contractions
    test[feature] = test[feature].apply(lambda x: map_contraction(x))
    # Trim text
    test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

# Geting host count
train = pd.read_csv('/kaggle/input/google-quest-challenge/train.csv')
N_CLASS_HOST = train['host'].nunique()

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output_seq = Dense(N_CLASS, activation="sigmoid", name="output_seq")(x)
    output_class = Dense(N_CLASS_HOST, activation="softmax", name="output_class")(pooled_output)
    output = Concatenate(name="output")([output_seq, output_class])

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test)[: ,:N_CLASS] / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.921081,0.629362,0.171062,0.464114,0.617284,0.593341,0.680471,0.643727,0.653579,...,0.905122,0.887826,0.551318,0.958023,0.961771,0.805855,0.112773,0.049693,0.842038,0.906281
1,46,0.891092,0.534340,0.005201,0.773958,0.754090,0.925658,0.567729,0.433485,0.022932,...,0.728203,0.964927,0.661901,0.977709,0.988835,0.908605,0.944096,0.079698,0.071117,0.908729
2,70,0.911260,0.715843,0.039731,0.798284,0.822078,0.935612,0.566180,0.488591,0.078878,...,0.877223,0.941240,0.566057,0.978084,0.976484,0.894257,0.162222,0.054413,0.879168,0.936116
3,132,0.853093,0.407662,0.008550,0.657473,0.754420,0.929091,0.542246,0.403190,0.110625,...,0.653650,0.949780,0.633570,0.963239,0.983725,0.896706,0.864509,0.148029,0.426551,0.886194
4,200,0.933315,0.450045,0.048467,0.863200,0.714340,0.838271,0.650584,0.582211,0.069506,...,0.684567,0.921014,0.634074,0.973983,0.973672,0.840219,0.226722,0.118189,0.678159,0.926569


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.879474,0.577500,0.032598,0.708835,0.797564,0.868070,0.585914,0.464975,0.213423,...,0.777062,0.938755,0.652391,0.962059,0.975977,0.875927,0.553148,0.124406,0.515545,0.911429
std,2812.670060,0.055559,0.137356,0.058435,0.120674,0.110083,0.125681,0.048957,0.086900,0.194702,...,0.097008,0.023227,0.047844,0.012764,0.010910,0.040300,0.324524,0.068587,0.291938,0.021306
min,39.000000,0.697186,0.283665,0.002864,0.184260,0.251270,0.274597,0.470179,0.305217,0.004001,...,0.494695,0.838672,0.524050,0.905752,0.904490,0.735893,0.008343,0.007598,0.014009,0.818774
25%,2572.000000,0.842799,0.459897,0.007743,0.633611,0.749833,0.860277,0.548897,0.402054,0.068004,...,0.694432,0.925523,0.619216,0.955171,0.971347,0.853566,0.209309,0.070656,0.258994,0.898686
50%,5093.000000,0.883421,0.567860,0.011774,0.712762,0.813590,0.913905,0.578204,0.443025,0.140883,...,0.785517,0.943563,0.654251,0.964188,0.978095,0.882083,0.644103,0.124560,0.505630,0.914417
75%,7482.000000,0.924575,0.689575,0.024763,0.790194,0.871516,0.942509,0.618437,0.509261,0.317477,...,0.860973,0.956133,0.685055,0.971360,0.983883,0.905249,0.844045,0.170113,0.786932,0.926806
max,9640.000000,0.977577,0.903367,0.544551,0.977183,0.984020,0.988248,0.740875,0.732054,0.797597,...,0.960360,0.980703,0.819897,0.985365,0.993366,0.969136,0.978077,0.457961,0.993609,0.956312
